<center><h1>C2: Monty Hall Problem</h1></center>

The Monty Hall Problem (https://en.wikipedia.org/wiki/Monty_Hall_problem) is a very famous problem in Probability Theory. 


The problem can be described as follow:

Suppose that you are on a game show where you can win a car if you find behind which door it is. There are three doors ("\{0,1,2\}") and only behind one of them there is a car; behind the others, there is a goat. 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Monty_open_door.svg/1000px-Monty_open_door.svg.png" style="width:600px"/>

You have to options to pick a door. Firsly, you have to decide among all three doors. You pick one of them, say Door #1, and the host, who knows where the car is, opens another door, say Door #0, which has a goat. Then you have the option to choose again. The host asks to you: "Do you want to keep Door #1 or would you prefer changing to Door #2?"

The question is therefore: according to the available information, should the player change her mind?

<h2>A PGM's answer:</h2>

Let us model this problem by means of a Bayesian network:

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

We can identify 3 different random variables:

- Prize position: $ P \in \{0, 1, 2 \} $

- Player's Answer: $ A \in \{0, 1, 2\} $

- Host's choice: $ H \in \{0, 1, 2\} $

where each possible value in $P$, $A$ and $H$ identifies a door. 

We can safely assume that the prize has been put behind a door without any specific criteria. Therefore, the probability distribution is uniform:

In [ ]:
cpd_p = TabularCPD('P', 3, [[-0.0, -0.0, -0.0]]) # Set the probability values that correspond


Similarly, the player, without any knowledge, is probably making her first choice randomly:


In [ ]:
cpd_a = TabularCPD('A', 3, [[-0.0, -0.0, -0.0]]) # Set the probability values that correspond


Note that the host makes his decision when the player already said her first choice (and knowing where the car is). Thus, his choice depends on both the prize position and the player's choice:

<img src="images/bn_monty_hall.png"/>


In [ ]:
# Defining the network structure
model = BayesianModel([('A', 'H'), ('P', 'H')]) # A is a parent of H, P is a parent of H

The probability distribution of the remaining variable, the one modeling the behavior of the host can be inferred from the structure of the problem. The host chooses a door different from the player which does not have the prize and opens it. With this information, one can complete the table of probabilities-parameters of this BN:

<pre>
+---+---+---+----------+
| A | P | H | p(H|A,P) |
+---+---+---+----------+
| 0 | 0 | 0 | 0.0      |
| 0 | 0 | 1 | 0.5      |
| 0 | 0 | 2 | 0.5      |
+---+---+---+----------+
| 0 | 1 | 0 | 0.0      |
| 0 | 1 | 1 | 0.0      |
| 0 | 1 | 2 | 1.0      |
+---+---+---+----------+
| 0 | 2 | 0 | 0.0      |
| 0 | 2 | 1 | 1.0      |
| 0 | 2 | 2 | 0.0      |
+---+---+---+----------+
| A | P | H | p(H|A,P) |
+---+---+---+----------+
| 1 | 0 | 0 | 0.0      |
| 1 | 0 | 1 | 0.0      |
| 1 | 0 | 2 | 1.0      |
+---+---+---+----------+
| 1 | 1 | 0 | 0.5      |
| 1 | 1 | 1 | 0.0      |
| 1 | 1 | 2 | 0.5      |
+---+---+---+----------+
| 1 | 2 | 0 | 1.0      |
| 1 | 2 | 1 | 0.0      |
| 1 | 2 | 2 | 0.0      |
+---+---+---+----------+
| A | P | H | p(H|A,P) |
+---+---+---+----------+
| 2 | 0 | 0 | 0.0      |
| 2 | 0 | 1 | 1.0      |
| 2 | 0 | 2 | 0.0      |
+---+---+---+----------+
| 2 | 1 | 0 | 1.0      |
| 2 | 1 | 1 | 0.0      |
| 2 | 1 | 2 | 0.0      |
+---+---+---+----------+
| 2 | 2 | 0 | 0.5      |
| 2 | 2 | 1 | 0.5      |
| 2 | 2 | 2 | 0.0      |
+---+---+---+----------+
</pre>

In [ ]:
# Defining the CPDs:
cpd_h = TabularCPD('H', 3, [[0.0, 0.0, 0.0, 0.0, 0.5, 1.0, -0.0, -0.0, -0.0], #complete the last part of the table
                            [0.5, 0.0, 1.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0], 
                            [0.5, 1.0, 0.0, 1.0, 0.5, 0.0, -0.0, -0.0, -0.0]],
                  evidence=['A', 'P'], evidence_card=[3, 3])

# Associating the CPDs with the network structure
model.add_cpds(cpd_a, cpd_p, cpd_h)


Once the BN is built, we can check if everything is ok:


In [ ]:
# Some other methods
print(model.get_cpds())

# check the model structure and the associated CPD 
print(model.check_model())


Now we have modeled our world. Let's see what happens when we put it to work:

Let's say that the player selects Door #0 and, in turn, the host opens Door #2. Basic intuition probably says that the player has again a choice among two possible choices, and these are equally probable to hide the prize. But, is this intuition right? 

Let's calculate the probability of the prize location, i.e., $P(P | A=0, H=2)$. To do so, we need to use inference techniques:

In [ ]:
# Infering the posterior probability 
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)

posterior_p = infer.query(['P'], evidence={'A': 0, 'H': 2})

print(posterior_p['P'])

Note that the posterior probability of having the prize behind Door #1 is larger than behind Door #0. In this problem, if we take into account all the available information (including that given when the host selects a specific door), we can see that the best possible action is to change the initially selected door.

You can try changing the values of $A$ and $H$ (with the only condition that they cannot be the same) and you will realize that this conclusion holds for all the possible cases.

<hr/>

One final <b>question</b>, regarding the different types of reasoning pattern studied in this subject, which one fits better with the kind of reasoning that we are implementing in this solution for the Monty Hall problem?